In [2]:
def get_bio_probabilities(cell_type="HT-29", time_step_hours=1.0, cell_size_um=20.0):
    """
    Calcula probabilidades (muerte, migración y reproducción) 
    a partir de parámetros experimentales o extraídos de reportes.

    Args:
        cell_type : str
            Tipo de célula (por ejemplo: 'HT-29', 'HeLa', 'Mv1Lu').
        time_step_hours : float
            Cuántas horas reales representa un paso de simulación.
        cell_size_um : float
            Tamaño promedio de una célula en micras (µm).

    Returns:
        tuple
            (p_dead, p_migra, p_repro), todas entre 0 y 1.
    """

    try:
        # Validaciones para que el cálculo tenga sentido. 
        if time_step_hours <= 0:
            raise ValueError(
                "time_step_hours debe ser mayor que 0 (no tiene sentido un paso de tiempo negativo)."
            )
        if cell_size_um <= 0:
            raise ValueError(
                "cell_size_um debe ser mayor que 0 (el tamaño de la célula no puede ser cero o negativo)."
            )

        
        # Base de datos de parámetros biológicos
            # 'v_growth': Velocidad de crecimiento radial. 
            # 'doubling_time': Estimación estándar del ciclo celular para células de mamíferos. 
        stats = {
            "HT-29": {"v_growth": 1.93, "doubling_time": 24.0},
            "HeLa":  {"v_growth": 1.34, "doubling_time": 24.0},
            "Mv1Lu": {"v_growth": 11.50, "doubling_time": 24.0}
        }

        # Si el tipo de célula no existe, se usa HT-29 como "comodín".
        if cell_type not in stats:
            print(f"[Aviso] cell_type='{cell_type}' no encontrado. Se usará 'HT-29' como valor por defecto.")
        data = stats.get(cell_type, stats["HT-29"])

        # Probabilidad de reproducción (P_repro)
        # Aproximación: si la célula se duplica en 'doubling_time' horas,
        # entonces cada hora tiene probabilidad ~ 1/doubling_time de reproducirse.
        p_repro = time_step_hours / data["doubling_time"]
        p_repro = min(1.0, p_repro) 

        # Probabilidad de migración (P_migra)
        # Calculamos cuántas micras 'avanza' en este paso de tiempo.
        avance_um = data["v_growth"] * time_step_hours

        # La probabilidad se toma como fracción del tamaño de una célula.
        p_migra = avance_um / cell_size_um
        p_migra = min(1.0, p_migra)

        # Probabilidad de muerte (P_dead)
        # Regla simple: una fracción pequeña de la probabilidad de reproducción.
        p_dead = 0.05 * p_repro

        # Normalización opcional 
        # Mecanismo de seguridad para garantizar las probabilidades nunca rompan las leyes matemáticas
        total = p_repro + p_migra + p_dead
        if total > 1.0:
            p_repro /= total
            p_migra /= total
            p_dead /= total 

        return p_dead, p_migra, p_repro

    except Exception as e:
        # Cualquier error interno lo metemos RuntimeError
        raise RuntimeError(f"Error al calcular probabilidades biológicas para '{cell_type}': {e}") from e

In [ ]:
# Se piden los datos al usuario
if __name__ == "__main__":
    print("Cálculo de probabilidades biológicas")

    try:
        # Tipo de célula
        cell_type_input = input("Tipo de célula [HT-29 / HeLa / Mv1Lu] (por defecto HT-29): ").strip()
        if cell_type_input == "":
            cell_type_input = "HT-29"

        # Paso de tiempo en horas
        dt_str = input("Paso de tiempo en horas (ej. 2.0): ").strip()
        time_step_hours = float(dt_str)

        # Tamaño de la célula en micras
        size_str = input("Tamaño de la célula en µm (ej. 20.0): ").strip()
        cell_size_um = float(size_str)

        # Función
        P_dead, P_migra, P_repro = get_bio_probabilities(
            cell_type=cell_type_input,
            time_step_hours=time_step_hours,
            cell_size_um=cell_size_um
        )

        # Resultados
        print("\n--- Resultados ---")
        print(f"Tipo de célula: {cell_type_input}")
        print(f"Paso de tiempo: {time_step_hours} h")
        print(f"Tamaño de célula: {cell_size_um} µm\n")

        print(f"P_repro = {P_repro:.4f}  # Probabilidad de que la célula se divida en este paso")
        print(f"P_migra = {P_migra:.4f}  # Probabilidad de que la célula migre")
        print(f"P_dead  = {P_dead:.4f}  # Probabilidad de que la célula muera en este paso")

    except ValueError as e:
        # Errores típicos de conversión de texto a float
        print(f"[Error] Alguno de los valores numéricos no es válido: {e}")
    except RuntimeError as e:
        # Errores internos de la función
        print(f"[Error] {e}")
    except Exception as e:
        # Cualquier otra cosa inesperada
        print(f"[Error inesperado] {e}")

Cálculo de probabilidades biológicas
